DESAFIO ENGENHEIRO DE DADOS JÚNIOR - STILINGUE

In [ ]:
# importando bibliotecas

import requests # responsável por enviar requisições HTTP
import json
import time
import pandas as pd
import numpy as np
from requests.adapters import HTTPAdapter 
from requests.packages.urllib3.util.retry import Retry

Fazendo as requisições e construindo a base de dados de localidades. 

In [ ]:
request = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/municipios') # faz a requisição
municipios = json.loads(request.content) # acessar os atributos em formato json

qnt = (len(municipios))

dados = []

# fazendo a coleta das informações necessárias na api de localidade 

for i in range(qnt):
    #print(municipios[i])
    print(municipios[i]['id'], '-', 
          municipios[i]['nome'], '-', 
          municipios[i]['microrregiao']['mesorregiao']['UF']['id'], '-', 
          municipios[i]['microrregiao']['mesorregiao']['UF']['sigla'], '-', 
          municipios[i]['microrregiao']['mesorregiao']['UF']['nome'], '-',
          municipios[i]['microrregiao']['mesorregiao']['UF']['regiao']['id'], '-',
          municipios[i]['microrregiao']['mesorregiao']['UF']['regiao']['nome'])

    dados.append(municipios[i]['id']) # id localidade
    dados.append(municipios[i]['nome']) # nome município
    dados.append(municipios[i]['microrregiao']['mesorregiao']['UF']['id']) # id UF
    dados.append(municipios[i]['microrregiao']['mesorregiao']['UF']['sigla']) # sigla UF
    dados.append(municipios[i]['microrregiao']['mesorregiao']['UF']['nome']) # nome UF
    dados.append(municipios[i]['microrregiao']['mesorregiao']['UF']['regiao']['id']) #id região
    dados.append(municipios[i]['microrregiao']['mesorregiao']['UF']['regiao']['nome']) # nome região


dados

Organizando a base e transformando-a em um dataframe pra ficar melhor apresentado.

In [ ]:
matriz_np = np.array(dados)
matriz_ajustada = np.reshape(matriz_np, (qnt,7))
#matriz_ajustada

df = pd.DataFrame()

colunas = ['id_localidade', 'nome_localidade', 'id_uf', 'sigla_uf', 'nome_uf', 'id_regiao', 'nome_regiao']

df = pd.DataFrame(matriz_ajustada, columns=colunas)

df

In [ ]:
df.to_csv('localidade.csv', encoding='latin-1', index=False) # salvando a base de localidade em csv

Fazendo as requisições e construindo a base de dados dos top 10 nomes no Brasil.

In [ ]:
request_ranking = requests.get(f'https://servicodados.ibge.gov.br/api/v2/censos/nomes/ranking')
request_nomes = requests.get(f'https://servicodados.ibge.gov.br/api/v2/censos/nomes')

ranking = json.loads(request_ranking.content)
nomes = json.loads(request_nomes.content)

dados_top10_BR = []

for i in range(10):
    print(nomes[i]['nome'], '-',
          nomes[i]['freq'])


    dados_top10_BR.append(nomes[i]['nome'])
    dados_top10_BR.append(nomes[i]['freq'])

dados_top10_BR

Organizando a base e transformando-a em um dataframe pra ficar melhor apresentado.

In [ ]:
matriz_np = np.array(dados_top10_BR)
matriz_ajustada = np.reshape(matriz_np, (10,2))
#matriz_ajustada

df_top10 = pd.DataFrame()

colunas = ['Nome', 'Frequencia']

df_top10 = pd.DataFrame(matriz_ajustada, columns=colunas)

df_top10

In [ ]:
df_top10.to_csv('top10.csv', encoding='latin-1', index=False) # salvando a base dos top 10 nomes Brasil em csv

In [ ]:
def buscar_nomes_top10_Local(local, nome):
    
    retry_strategy = Retry(
        total = 3, # número total de tentativas de repetição
        status_forcelist = [429, 500, 502, 503, 504], # Prevendo possíveis erros do servidor 
        method_whitelist = ["GET"], # para realizar a coleta de informações eu só preciso utilizar o get na nova requisição
        backoff_factor = 120 # define quanto tempo o processo fica inativo entre as solicitações com falha e a nova requisição
    )
    adapter = HTTPAdapter(max_retries = retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)
    
    request_ranking = http.get(f'https://servicodados.ibge.gov.br/api/v2/censos/nomes/ranking?localidade={local}')
    request_nomes = http.get(f'https://servicodados.ibge.gov.br/api/v2/censos/nomes/{nome}?localidade={local}')

    ranking = json.loads(request_ranking.content)
    nomes = json.loads(request_nomes.content)
    
    for i in range(len(nomes)): 
        for j in range(len(nomes[i]['res'])):
            top10 = []
            print(nomes[i]['nome'], '-',
                  nomes[i]['localidade'], '-')
            top10.append(nomes[i]['nome'])
            top10.append(nomes[i]['localidade'])
            print(nomes[i]['res'][j]['periodo'], '-',
                  nomes[i]['res'][j]['frequencia'],)
            top10.append(nomes[i]['res'][j]['periodo'])
            top10.append(nomes[i]['res'][j]['frequencia'])
                
            dados_top10_localidade.append(top10)

    dados_top10_localidade

In [ ]:
if __name__ == '__main__':
    dados_top10_localidade = []
    for i in range(len(municipios)):
        print(i)
        for j in range(10):
            buscar_nomes_top10_Local(df['ID Localizacao'][i], df_top10['Nome'][j])

In [ ]:
dados_top10_localidade

In [ ]:
matriz_np = np.array(dados_top10_localidade)
#matriz_np
matriz_ajustada = np.reshape(matriz_np, (len(dados_top10_localidade), 4))
#matriz_ajustada

colunas = ['nome', 'id_localidade', 'periodo', 'frequencia']

df_nomes = pd.DataFrame(matriz_ajustada, columns=colunas)

In [ ]:
df_nomes

In [ ]:
df_nomes.to_csv('nomes_localidade.csv', encoding='latin-1', index=False)

In [ ]:
#--------------------------------------------------